In [1]:
"""-------------------------------clone repository-------------------------------"""


#!git clone https://huggingface.co/google-bert/bert-base-uncased
!git clone https://huggingface.co/datasets/stanfordnlp/imdb

Cloning into 'imdb'...
remote: Enumerating objects: 63, done.
remote: Total 63 (delta 0), reused 0 (delta 0), pack-reused 63 (from 1)
Unpacking objects: 100% (63/63), 12.91 KiB | 777.00 KiB/s, done.
Filtering content: 100% (3/3), 79.58 MiB | 13.82 MiB/s, done.


In [ ]:
"""-------------------------------清理内存-------------------------------"""


import gc
import torch
import os
#del model
os._exit(00)


In [1]:
"""-------------------------------定义训练模式、分类器-------------------------------"""


MODEL_NAME = "bert-base-uncased"
MODE = True # False for freezing,True otherwise

from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
import pandas as pd


train_data = pd.read_parquet("/content/imdb/plain_text/train-00000-of-00001.parquet")

train_text = tokenizer(train_data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
train_label = torch.tensor(train_data['label'].tolist())

test_data = pd.read_parquet("/content/imdb/plain_text/test-00000-of-00001.parquet")

test_text = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
test_label = torch.tensor(test_data['label'].tolist())

from transformers import BertModel, BertTokenizer, BertForSequenceClassification,AdamW
import torch
from torch import nn

from transformers import BertConfig


class BertTextClassificationModel(nn.Module):
    def __init__(self, num_classes=2, model_name=MODEL_NAME, dropout_prob=0.1,mode=MODE):
        super(BertTextClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_prob)  # 添加dropout层以防止过拟合
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(256, num_classes)
        )

        # 冻结BERT模型的参数
        for param in self.bert.parameters():
            param.requires_grad = mode

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [43]:
"""-------------------------------训练分类器-------------------------------"""



from transformers import BertModel, BertTokenizer, BertForSequenceClassification,AdamW
import torch
from torch import nn

from transformers import BertConfig

# 实例化模型
model = BertTextClassificationModel(num_classes=2)
# 定义优化器和损失函数
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

from torch.utils.data import TensorDataset, DataLoader




batch_size= 32
# 训练模型
model.train()
if torch.cuda.is_available():
    device = torch.device("cuda")  # 如果有GPU，设备类型设置为"cuda"
else:
    device = torch.device("cpu")   # 如果没有GPU，设备类型设置为"cpu"
model.to(device)


# Create a dataset
train_dataset = TensorDataset(train_text['input_ids'], train_text['attention_mask'], train_label)

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
import os
for epoch in range(3):
    print("epoch:",epoch)
    count = 0
    loss_list=[]
    for data in train_dataloader:
        #torch.cuda.empty_cache()
        #model.to(device)
        count = count+batch_size
        input, attention_mask,label =  data
        input = input.to(device)
        label = label.to(device)
        attention_mask = attention_mask.to(device)
        optimizer.zero_grad()
        outputs = model(input,attention_mask)
        #print(outputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        print(f"count {count}, Loss: {loss.item()}")
        loss_list.append(loss.item())
    with open(f'model_{MODEL_NAME}_{MODE}_{epoch+1}_loss.txt', 'w') as file:
      for item in loss_list:
          file.write(f"{item}\n")
    torch.save(model.state_dict(), f'model_{MODEL_NAME}_{MODE}_{epoch+1}.pth')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch: 0
count 32, Loss: 0.6818029880523682
count 64, Loss: 0.7073661088943481
count 96, Loss: 0.6769632697105408
count 128, Loss: 0.7083679437637329
count 160, Loss: 0.6946147084236145
count 192, Loss: 0.6818351745605469
count 224, Loss: 0.6617627143859863
count 256, Loss: 0.692987322807312
count 288, Loss: 0.6859220266342163
count 320, Loss: 0.6529792547225952
count 352, Loss: 0.65240478515625
count 384, Loss: 0.6589126586914062
count 416, Loss: 0.6040855646133423
count 448, Loss: 0.5711455941200256
count 480, Loss: 0.623327910900116
count 512, Loss: 0.5696452856063843
count 544, Loss: 0.6719508171081543
count 576, Loss: 0.543320894241333
count 608, Loss: 0.7647387385368347
count 640, Loss: 0.6111541390419006
count 672, Loss: 0.5128993391990662
count 704, Loss: 0.7197791934013367
count 736, Loss: 0.7881654500961304
count 768, Loss: 0.705785870552063
count 800, Loss: 0.6581024527549744
count 832, Loss: 0.6364842653274536
count 864, Loss: 0.6513118147850037
count 896, Loss: 0.727427840

In [63]:
"""-------------------------------在测试集上测试训练好的checkpoint-------------------------------"""



import torch.nn.functional as F
import gc

acc_list = []
loss_list = []
#test_data = pd.read_parquet("/content/imdb/plain_text/test-00000-of-00001.parquet")

#test_text = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
#test_label = torch.tensor(test_data['label'].tolist())
for i in range(3):
  model = BertTextClassificationModel(num_classes=2)
  model.load_state_dict(torch.load(f'model_{MODEL_NAME}_{MODE}_{i+1}.pth'))

  model.eval()

  # Create a dataset
  test_dataset = TensorDataset(test_text['input_ids'], test_text['attention_mask'], test_label)

  # DataLoader
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
  if torch.cuda.is_available():
      device = torch.device("cuda")  # 如果有GPU，设备类型设置为"cuda"
  else:
      device = torch.device("cpu")   # 如果没有GPU，设备类型设置为"cpu"
  model.to(device)



  # Put the model in evaluation mode
  model.eval()

  # Initialize variables to keep track of metrics
  total = 0
  correct = 0
  total_loss = 0

  # No gradient computation needed during inference
  with torch.no_grad():
      for batch in test_dataloader:
          input_ids, attention_masks, labels = batch
          input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

          # Forward pass, get predictions
          outputs = model(input_ids, attention_mask=attention_masks)

          # Calculate loss if you want to report it
          loss = criterion(outputs, labels)
          total_loss += loss.item()

          # Convert model logits to class predictions
          predictions = torch.argmax(outputs, dim=1)

          # Update correct prediction count
          correct += (predictions == labels).sum().item()
          total += labels.size(0)
          print(total)

  # Calculate accuracy
  accuracy = correct / total
  model = None
  gc.collect()
  torch.cuda.empty_cache()
  acc_list.append(accuracy)
  loss_list.append(total_loss/len(test_dataloader))
  print(f"Accuracy on test data: {accuracy*100:.2f}%")
  print(f"Average loss on test data: {total_loss/len(test_dataloader):.2f}")
print(acc_list)
print(loss_list)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
"""---------------------------查找pretrain分类失败而finetune分类成功的样本-------------------------------"""



from torch.utils.data import TensorDataset, DataLoader



if torch.cuda.is_available():
    device = torch.device("cuda")  # 如果有GPU，设备类型设置为"cuda"
else:
    device = torch.device("cpu")   # 如果没有GPU，设备类型设置为"cpu"


model_pretrained = BertTextClassificationModel(num_classes=2,mode=False)
model_pretrained.load_state_dict(torch.load('/content/model_bert-base-uncased_False_2.pth'))
model_finetuned = BertTextClassificationModel(num_classes=2,mode=True)
model_finetuned.load_state_dict(torch.load('/content/model_bert-base-uncased_True_2.pth'))

model_pretrained.to(device)
model_pretrained.eval()
model_finetuned.to(device)
model_finetuned.eval()

test_dataset = TensorDataset(test_text['input_ids'], test_text['attention_mask'], test_label)

# DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=1,shuffle=False)
count = -1
num = 0
with torch.no_grad():
    for batch in test_dataloader:
        if(num==2 and count<13000):
          count+=1
          continue
        if(num==4):
          break
        count+=1
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        # Forward pass, get predictions
        outputs_pretrained = model_pretrained(input_ids, attention_mask=attention_masks)


        # Convert model logits to class predictions
        predictions = torch.argmax(outputs_pretrained, dim=1)

        # Update correct prediction count
        correct_pretrained = (predictions == labels).sum().item()

                # Forward pass, get predictions
        outputs_finetuned = model_finetuned(input_ids, attention_mask=attention_masks)


        # Convert model logits to class predictions
        predictions = torch.argmax(outputs_finetuned, dim=1)

        # Update correct prediction count
        correct_finetuned = (predictions == labels).sum().item()
        if(correct_pretrained==0 and correct_finetuned==1):
          print(count)
          print(input_ids)
          num += 1

6
tensor([[  101,  7527, 13109,  5686, 26730,  2038,  2081,  2070,  1997,  1996,
          2190,  2530,  7761,  2840,  2895,  5691,  2412,  2550,  1012,  1999,
          3327,  2149, 13945,  1016,  1010,  3147, 11203,  1010,  2569,  2749,
          1998,  6151,  2483, 29462,  1016,  2024,  2035,  2895, 10002,  1012,
          2017,  2064,  2425,  7527,  2038,  1037,  2613,  6896,  2005,  1996,
          6907,  1998,  2010,  3152,  2024,  2467,  2724,  3993,  1010,  5541,
          1998,  4629,  3821,  1010,  2007,  2070,  1997,  1996,  2190,  2954,
         10071,  2019,  2895,  5470,  2071,  3246,  2005,  1012,  1999,  3327,
          2002,  2038,  2179,  1037, 18437,  2007,  3660,  4748, 14322,  1010,
          2004, 10904,  2019,  3364,  1998,  2895,  9256,  2004,  2017,  2071,
          3246,  2005,  1012,  2023,  2003, 15356,  2041,  2007,  2569,  2749,
          1998,  6151,  2483, 29462,  1016,  1010,  2021,  6854,  1996, 11133,
          2074,  2987,  1005,  1056,  2444,  2039,

In [11]:
"""-------------------------------test-1-------------------------------"""



text = """When I first saw a glimpse of this movie, I quickly noticed the actress who was playing the role of Lucille Ball. Rachel York's portrayal of Lucy is absolutely awful. Lucille Ball was an astounding comedian with incredible talent. To think about a legend like Lucille Ball being portrayed the way she was in the movie is horrendous. I cannot believe out of all the actresses in the world who could play a much better Lucy, the producers decided to get Rachel York. She might be a good actress in other roles but to play the role of Lucille Ball is tough. It is pretty hard to find someone who could resemble Lucille Ball, but they could at least find someone a bit similar in looks and talent. If you noticed York's portrayal of Lucy in episodes of I Love Lucy like the chocolate factory or vitavetavegamin, nothing is similar in any way-her expression, voice, or movement.<br /><br />To top it all off, Danny Pino playing Desi Arnaz is horrible. Pino does not qualify to play as Ricky. He's small and skinny, his accent is unreal, and once again, his acting is unbelievable. Although Fred and Ethel were not similar either, they were not as bad as the characters of Lucy and Ricky.<br /><br />Overall, extremely horrible casting and the story is badly told. If people want to understand the real life situation of Lucille Ball, I suggest watching A&E Biography of Lucy and Desi, read the book from Lucille Ball herself, or PBS' American Masters: Finding Lucy. If you want to see a docudrama, "Before the Laughter" would be a better choice. The casting of Lucille Ball and Desi Arnaz in "Before the Laughter" is much better compared to this. At least, a similar aspect is shown rather than nothing."""
token = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
print(token)

{'input_ids': tensor([[  101,  2043,  1045,  2034,  2387,  1037, 12185,  1997,  2023,  3185,
          1010,  1045,  2855,  4384,  1996,  3883,  2040,  2001,  2652,  1996,
          2535,  1997, 28016,  3608,  1012,  5586,  2259,  1005,  1055, 13954,
          1997,  7004,  2003,  7078,  9643,  1012, 28016,  3608,  2001,  2019,
          2004, 24826, 15683,  9971,  2007,  9788,  5848,  1012,  2000,  2228,
          2055,  1037,  5722,  2066, 28016,  3608,  2108,  6791,  1996,  2126,
          2016,  2001,  1999,  1996,  3185,  2003,  7570, 14343, 15482,  2271,
          1012,  1045,  3685,  2903,  2041,  1997,  2035,  1996, 19910,  1999,
          1996,  2088,  2040,  2071,  2377,  1037,  2172,  2488,  7004,  1010,
          1996,  6443,  2787,  2000,  2131,  5586,  2259,  1012,  2016,  2453,
          2022,  1037,  2204,  3883,  1999,  2060,  4395,  2021,  2000,  2377,
          1996,  2535,  1997, 28016,  3608,  2003,  7823,  1012,  2009,  2003,
          3492,  2524,  2000,  2424,  

In [ ]:
"""-------------------------------test-2-------------------------------"""



MODEL_NAME = "bert-base-uncased"

from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
import pandas as pd
model=BertModel.from_pretrained(MODEL_NAME)
output=model(input_ids=[1,0],attention_mask=[1,1])
print(output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


AttributeError: 'list' object has no attribute 'size'

In [ ]:
"""-------------------------------test-3-------------------------------"""



"""inference by pretrained or finetuned"""

MODEL_NAME = "bert-base-uncased"
import pandas as pd

# 代码示例：加载预训练的BERT模型
from transformers import BertTokenizer, BertModel
import torch

# 加载BERT的tokenizer和预训练模型
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)


train_data = pd.read_parquet("/content/imdb/plain_text/train-00000-of-00001.parquet")

train_text = tokenizer(train_data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
train_label = torch.tensor(train_data['label'].tolist())

test_data = pd.read_parquet("/content/imdb/plain_text/test-00000-of-00001.parquet")

test_text = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
test_label = torch.tensor(test_data['label'].tolist())